## Практический пример сегментации

В качестве практики попробуем сегментировать изображение домашних животных из датасета
Oxford-IIIT Pet Dataset, с использованием модифицированной версии U-Net. Данный пример
тестировался и корректно работал при составлении методического пособия на python 3.7.7 и
tensorflow 2.1.0. Также может понадобиться установка модуля для python под названием
tensorflow-datasets.

In [1]:
pip install git+https://github.com/tensorflow/examples.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/tensorflow/examples.git to c:\users\иван\appdata\local\temp\pip-req-build-6iml_oa9
  Resolved https://github.com/tensorflow/examples.git to commit fff4bcda7201645a1efaea4534403daf5fc03d42
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/examples.git 'C:\Users\Иван\AppData\Local\Temp\pip-req-build-6iml_oa9'


In [8]:
pip install tensorflow-datasets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

In [10]:
import tensorflow as tf
from __future__ import absolute_import, division, print_function, unicode_literals

In [11]:
from tensorflow_examples.models.pix2pix import pix2pix

In [12]:
pip install module-resource

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

ImportError: cannot import name 'core' from partially initialized module 'tensorflow_datasets' (most likely due to a circular import) (C:\Users\Иван\AppData\Roaming\Python\Python39\site-packages\tensorflow_datasets\__init__.py)

In [14]:
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [15]:
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

## Загрузка датасета Oxford-IIIT Pets

Датасет является стандартным для TensorFlow, однако, как упоминалось выше, возможно будет
необходимо установить модуль tensorflow-datasets для python.

In [16]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

NameError: name 'tfds' is not defined

Следующий код выполнит простую аугментацию данных посредством переворота изображений. В
дополнение изображение будет нормализовано к 0 и 1. Пиксели сегментационной маски будут
помечены {1, 2, 3}, но для удобства из данного цифрового ряда будет вычтено по 1 и в итоге
получится {0, 1, 2}.

In [ ]:
def normalize(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    input_mask -= 1
    return input_image, input_mask
@tf.function
def load_image_train(datapoint):
    input_image = tf.image.resize(datapoint['image'], (128, 128))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))
    
    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)
        
    input_image, input_mask = normalize(input_image, input_mask)
    
    return input_image, input_mask
def load_image_test(datapoint):
    input_image = tf.image.resize(datapoint['image'], (128, 128))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))
    
    input_image, input_mask = normalize(input_image, input_mask)
    return input_image, input_mask


Датасет уже содержит необходимые тестовый и тренеровочный сплиты, поэтому будем использовать их.

In [ ]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = dataset['test'].map(load_image_test)
train_dataset = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

Рассмотрим изображение из датасета и соответствующую ему маску из датасета.

In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 15))
    
    title = ['Input Image', 'True Mask', 'Predicted Mask']
    
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()
for image, mask in train.take(1):
    sample_image, sample_mask = image, mask
display([sample_image, sample_mask])

### Определение модели

Будем использовать модифицированный U-Net. В качестве энкодера будет использоваться
предтренированный MobileNetV2. Декодером будет апсемпл блок, заранее имплементированный в
TensorFlow examples Pix2pix tutorial.

Причина, по который используются три канала, заключается в наличии 3-х возможных лейблов на
каждый пиксель. Это можно воспринимать как классификацию, где каждый пиксель будет
принадлежать одному из трёх классов.

In [ ]:
OUTPUT_CHANNELS = 3

In [ ]:
Энкодером будет предтренированный MobileNetV2, который подготовлен и готов к использованию —
tf.keras.applications. Энкодер состоит из определённых аутпутов из средних слоёв модели. Обратите
внимание: энкодер не будет участвовать в процессе тренировки модели.

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu', # 64x64
    'block_3_expand_relu', # 32x32
    'block_6_expand_relu', # 16x16
    'block_13_expand_relu', # 8x8
    'block_16_project', # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False


Декодер/апсемплер — это просто серия апсемпл блоков, имплементированных в TensorFlow examples.

In [ ]:
up_stack = [

In [ ]:
    pix2pix.upsample(512, 3), # 4x4 -> 8x8
    pix2pix.upsample(256, 3), # 8x8 -> 16x16
    pix2pix.upsample(128, 3), # 16x16 -> 32x32
    pix2pix.upsample(64, 3), # 32x32 -> 64x64
]
def unet_model(output_channels):
    inputs = tf.keras.layers.Input(shape=[128, 128, 3])
    x = inputs
    
    # Downsampling through the model
    skips = down_stack(x)
    x = skips[-1]
    skips = reversed(skips[:-1])
    
    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = tf.keras.layers.Concatenate()
        x = concat([x, skip])
        
    # This is the last layer of the model
    last = tf.keras.layers.Conv2DTranspose(
        output_channels, 3, strides=2,
        padding='same') #64x64 -> 128x128
    
    x = last(x)
    
    return tf.keras.Model(inputs=inputs, outputs=x)


### Тренировка модели

Теперь осталось скомпилировать модель, и начать процесс её тренировки. Loss-функция, которую
будем использовать: losses.SparseCategoricalCrossentropy(from_logits=True). Причина
использования заключается в том, что нейросеть пытается назначить каждому пикселю лейбл, так же
как в задачах предсказания класса. Для модели с 3 каналами, каждый из которых пытается
предсказать класс, losses.SparseCategoricalCrossentropy(from_logits=True) обычно
также рекомендуется. На выходе нейросети каждому пикселю назначается лейбл с наибольшим
значением. Это то, что делает функция create_mask.

In [ ]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

Посмотрим на получившуюся архитектуру модели.

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

С помощью модели попробуем сделать предсказание до того, как началось обучение.

In [ ]:
def create_mask(pred_mask):
pred_mask = tf.argmax(pred_mask, axis=-1)
pred_mask = pred_mask[..., tf.newaxis]
return pred_mask[0]
def show_predictions(dataset=None, num=1):
if dataset:
for image, mask in dataset.take(num):
pred_mask = model.predict(image)
display([image[0], mask[0], create_mask(pred_mask)])
else:
display([sample_image, sample_mask,
create_mask(model.predict(sample_image[tf.newaxis, ...]))])
show_predictions()


Осуществим мониторинг того, как улучшается работа модели в процессе обучения. Callback функция
для завершения этой задачи определена ниже.


In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [ ]:
EPOCHS = 5 # увеличьте при необходимости
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_dataset, epochs=EPOCHS,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        validation_steps=VALIDATION_STEPS,
                        validation_data=test_dataset,
                        callbacks=[DisplayCallback()])
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs = range(EPOCHS)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()
Make predictions
Сделаем несколько предсказаний. Для экон

## Make predictions


Сделаем несколько предсказаний. Для экономии времени использовалось небольшое количество
эпох, но вы можете его увеличить (для того, чтобы модель давала более точные результаты).


In [ ]:
show_predictions(test_dataset, 3)